In [1]:
!pip install ssqueezepy mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 285.3 kB/s eta 0:00:00a 0:00:01


In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import cv2
import os
import tqdm.notebook as tq
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import random
from glob import glob
import scipy.io
import torch.nn as nn
import numpy as np
import os
import sys
import re
import pandas as pd
from skimage.transform import resize
import warnings
from pathlib import Path 
import pickle
import mne
from ssqueezepy import cwt

warnings.filterwarnings('ignore')

In [3]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
api_token = {"username":"****","key":"****"}
import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c alvi-hack-2023
!unzip alvi-hack-2023.zip 

100%|███████████████████████████████████████▊| 228M/229M [00:13<00:00, 22.1MB/s]
100%|████████████████████████████████████████| 229M/229M [00:13<00:00, 17.6MB/s]
Archive:  alvi-hack-2023.zip
  inflating: data_submission/00.npz  
  inflating: data_submission/01.npz  
  inflating: data_submission/02.npz  
  inflating: data_submission/03.npz  
  inflating: data_submission/04.npz  
  inflating: data_submission/05.npz  
  inflating: data_submission/06.npz  
  inflating: data_submission/07.npz  
  inflating: data_submission/08.npz  
  inflating: data_submission/09.npz  
  inflating: data_submission/10.npz  
  inflating: data_submission/11.npz  
  inflating: data_submission/12.npz  
  inflating: data_submission/13.npz  
  inflating: data_submission/14.npz  
  inflating: data_train/00.npz       
  inflating: data_train/01.npz       
  inflating: data_train/02.npz       
  inflating: data_train/03.npz       
  inflating: data_train/04.npz       
  inflating: data_train/05.npz       
  inflating

In [4]:
class GenBlock(nn.Module):
    def __init__(self, in_features, out_features, stride=2,padding=1, down=True, activation='relu', dropout=False):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_features, out_features, 4, stride, padding, bias=False, padding_mode='reflect') 
            if down else nn.ConvTranspose2d(in_features, out_features, 4, stride, padding, bias=False),
            nn.BatchNorm2d(out_features),
            nn.ReLU() if activation == 'relu' else nn.LeakyReLU(0.2)
        )
        self.use_dropout = dropout
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.conv(x)
        return x if not self.use_dropout else self.dropout(x)

In [7]:
class Generator(nn.Module):
    def __init__(self, in_channels=8, features=64):
        super().__init__()
        self.down1 = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, bias=False, padding_mode='reflect'),
            nn.LeakyReLU(0.2)
        ) #128
        
        self.down2 = GenBlock(features, 2*features, down=True, activation='leaky', dropout=False) #64
        self.down3 = GenBlock(2*features, 2*features, down=True, activation='leaky', dropout=False) #32
        self.down4 = GenBlock(2*features, 4*features, down=True, activation='leaky', dropout=False) #16
        self.down5 = GenBlock(4*features, 4*features,padding=0, down=True, activation='leaky', dropout=False) #8
        self.down6 = GenBlock(4*features, 8*features, down=True, activation='leaky', dropout=False) #4

        self.bottleneck = nn.Sequential(
            nn.Conv2d(8*features, 16*features, kernel_size=1, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(16*features, 8*features, kernel_size=1, stride=2, padding=1),
            nn.ReLU()
        )

        self.up1 = GenBlock(2*8*features, 4*features, down=False, activation='relu', dropout=True) #8
        self.up2 = GenBlock(2*4*features, 4*features,padding=0, down=False, activation='relu', dropout=True) #16
        self.up3 = GenBlock(2*4*features, 2*features, down=False, activation='relu', dropout=True) #32
        self.up4 = GenBlock(2*2*features, 2*features, down=False, activation='relu', dropout=True) #64
        self.up5 = GenBlock(2*2*features, features, down=False, activation='relu', dropout=True) #128
        self.up6 = nn.Sequential(
            nn.ConvTranspose2d(2*features, in_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        ) #256

    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(d1)

        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        bn = self.bottleneck(d6)
        u1 = self.up1(torch.cat([bn, d6], dim=1))
        u2 = self.up2(torch.cat([u1, d5], dim=1))
        u3 = self.up3(torch.cat([u2, d4], dim=1))
        u4 = self.up4(torch.cat([u3, d3], dim=1))
        u5 = self.up5(torch.cat([u4, d2], dim=1))
        u6 = self.up6(torch.cat([u5, d1], dim=1))
        return u6


In [8]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=2):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, stride, bias=False, padding_mode='reflect'),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )
    def forward(self, x):
        return self.conv(x)

In [9]:
class Descriminator(nn.Module):
    def __init__(self, in_channels=8):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=4, stride=2, padding=1, padding_mode='reflect'),
            nn.LeakyReLU(0.2)
        )
        self.model = nn.Sequential(
            CNNBlock(64, 128, stride=2),
            CNNBlock(128, 256, stride=2),
            CNNBlock(256, 256, stride=2),
            CNNBlock(256, 512, stride=2),
            CNNBlock(512, 512, stride=1),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1, padding_mode='reflect')

        )
        self.sigmoid = nn.Sigmoid()
        self.flatten = nn.Flatten()
    def forward(self, x, ):
        x = self.initial(x)
        return self.sigmoid(self.flatten(self.model(x)))

In [10]:
class DenDataset(Dataset):
    def __init__(self, path, window):
        self.window = window
        self.path = path
        self.x = torch.from_numpy(np.load(path + sorted(os.listdir(path))[0])['data_myo']).type(torch.FloatTensor)
        for i in sorted(os.listdir(path)):
            if i != sorted(os.listdir(path))[0]:
                self.x = torch.cat([self.x, torch.from_numpy(np.load(path + i)['data_myo']).type(torch.FloatTensor)], dim=0)
        x_add = self.x[0]
        for i in range(self.window):
            self.x = torch.cat([x_add[None], self.x], dim=0)
    def __len__(self):
        return (len(self.x) - 200) // 200

    def __getitem__(self, idx):
        x = self.x[idx:idx + self.window]
        x = x.permute(1,0)
        x=convertDF2MNE(x)
        Wx, scales = cwt(x[0], 'morlet')
        x=np.abs(Wx)
        return resize(x, (8, 224, 224))

In [11]:
train_path = '/kaggle/working/data_train/'



In [12]:
train_ds = DenDataset(train_path, 200)
loader = DataLoader(train_ds, batch_size=64, shuffle=True)


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)



cuda


In [14]:
discriminator = Descriminator().to(device)
generator = Generator().to(device)

In [15]:
lr = 0.0002

model = {
    "discriminator": discriminator,
    "generator": generator
}

criterion = {
    "discriminator": nn.BCELoss(),
    "generator": nn.BCELoss()
}

In [16]:

def convertDF2MNE(sub):
    info = mne.create_info([str(i) for i in range(len(sub))], ch_types=['eeg'] * len(sub), sfreq=200, verbose=False)
    data=mne.io.RawArray(sub, info, verbose=False)
    data.set_eeg_reference(verbose=False)
    epochs=mne.make_fixed_length_epochs(data,duration=1,overlap=0, verbose=False)
    return epochs._get_data(verbose=False)

In [17]:
def noisy(image):
    ch, row,col= image.shape
    mean = 0
    var = 0.1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(ch,row,col))
    gauss = gauss.reshape(ch,row,col)
    noisy = image + gauss
    return noisy

In [18]:
batch_size=64

In [19]:
def fit(model, criterion, epochs, lr):
    model["discriminator"].train()
    model["generator"].train()
  
    # Losses & scores
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []
  
    # Create optimizers
    optimizer = {
      "discriminator": torch.optim.Adam(model["discriminator"].parameters(), 
                                        lr=lr, betas=(0.5, 0.999)),
      "generator": torch.optim.Adam(model["generator"].parameters(),
                                    lr=lr, betas=(0.5, 0.999))
  }
  
    for epoch in range(epochs):
        loss_d_per_epoch = []
        loss_g_per_epoch = []
        real_score_per_epoch = []
        fake_score_per_epoch = []
        for real_images in tq.tqdm(loader):
          # Train discriminator
          # Clear discriminator gradients
            real_images = real_images.to(device)
            optimizer["discriminator"].zero_grad()

          # Pass real images through discriminator
            real_preds = model["discriminator"](real_images)
            real_targets = torch.ones(real_images.size(0), 1, device=device)
            real_loss = criterion["discriminator"](real_preds, real_targets)
            cur_real_score = torch.mean(real_preds).item()
          
          # Generate fake images
            for i in range(len(real_images)):
                real_images[i] = torch.from_numpy(noisy(real_images[i].cpu().detach().numpy()))
            latent = real_images
            fake_images = model["generator"](latent)

          # Pass fake images through discriminator
            fake_targets = torch.zeros(fake_images.size(0), 1, device=device)
            fake_preds = model["discriminator"](fake_images)
            fake_loss = criterion["discriminator"](fake_preds, fake_targets)
            cur_fake_score = torch.mean(fake_preds).item()

            real_score_per_epoch.append(cur_real_score)
            fake_score_per_epoch.append(cur_fake_score)

          # Update discriminator weights
            loss_d = real_loss + fake_loss
            loss_d.backward()
            optimizer["discriminator"].step()
            loss_d_per_epoch.append(loss_d.item())


          # Train generator
          # Clear generator gradients
            optimizer["generator"].zero_grad()
          
          # Generate fake images
            for i in range(len(real_images)):
                real_images[i] = torch.from_numpy(noisy(real_images[i].cpu().detach().numpy()))
            latent = real_images
            fake_images = model["generator"](latent)
          
          # Try to fool the discriminator
            preds = model["discriminator"](fake_images)
            targets = torch.ones(preds.shape[0], 1, device=device)
            loss_g = criterion["generator"](preds, targets)
          
          # Update generator weights
            loss_g.backward()
            optimizer["generator"].step()
            loss_g_per_epoch.append(loss_g.item())
          
      # Record losses & scores
        losses_g.append(np.mean(loss_g_per_epoch))
        losses_d.append(np.mean(loss_d_per_epoch))
        real_scores.append(np.mean(real_score_per_epoch))
        fake_scores.append(np.mean(fake_score_per_epoch))
      
      # Log losses & scores (last batch)
        print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
          epoch+1, epochs, 
          losses_g[-1], losses_d[-1], real_scores[-1], fake_scores[-1]))
  
  
    return losses_g, losses_d, real_scores, fake_scores

In [ ]:
%%time
history = fit(model, criterion, 25, lr)

In [17]:
losses_g, losses_d, real_scores, fake_scores = history

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(losses_d, '-')
plt.plot(losses_g, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses');

In [ ]:
plt.figure(figsize=(15, 6))

plt.plot(real_scores, '-')
plt.plot(fake_scores, '-')
plt.xlabel('epoch')
plt.ylabel('score')
plt.legend(['Real', 'Fake'])
plt.title('Scores');